In [35]:
import xml.etree.ElementTree as ET
from pylab import *
import networkx as nx
import pickle

In [36]:
def store(data, name):
    with open(name+'.p','wb') as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# Parsing the osm file using iterparse

In [32]:
def getxy(lat,lon):
    # gets X,Y xoordinates using latitude and longitude
    R = 6378000             # Radius of earth
    latr = lat*pi/180
    lonr = lon*pi/180
    x = R*cos(latr)*cos(lonr)
    y = R*cos(latr)*sin(lonr)
    return [x,y]

def node_attributes(elem):
    # Given a node element, gets its attributes like ID,latitude,longitude
    ID = elem.get('id')
    lat = float(elem.get('lat'))
    lon = float(elem.get('lon'))
    return ID,lat,lon

def way_attributes(elem):
    path = []
    ID = elem.get('id')
    for nd in elem:
        if nd.tag == "nd":
            path.append(nd.get('ref'))
    return ID,path  

def checkroad(way):
    for nd in way:
        if(nd.tag == 'tag'):
            if(nd.attrib['k'] == "highway"):
                return True
                break
    return False
        
def distance(x1,y1,x2,y2):
    return sqrt((x2-x1)**2 + (y2-y1)**2)
    
    
def parser(source):
    
    node = {}               # {'node_ID1' : [x1,y1], 'node_ID2' : [x2,y2],.... }
    node_data = {}          # same as node but with lat and lon instead of x,y
    way = {}                # {'way_ID1' : [nodes 1], 'way_ID2' : [nodes 2],.... }
    G = []                  # a list of edges - [(node1,node2,dist12),(node3,node4,dist34), ...]
        
    for event, elem in ET.iterparse(source):
        if elem.tag == "node":
            ID,lat,lon = node_attributes(elem)
            node[ID] = getxy(lat,lon)
            node_data[ID] = [lat,lon]
            elem.clear()
        
        if elem.tag == "way":
            if(not checkroad(elem)):
                continue
            ID,path = way_attributes(elem)
            for i in range(len(path)-1):
                [x1,y1] = node[path[i]]
                [x2,y2] = node[path[i+1]]
                d = distance(x1,y1,x2,y2)
                G.append((path[i],path[i+1],d)) # a list of tuples
            way[ID] = path
            elem.clear()
        
    return node_data,way,G

In [42]:
node_data,way,G = parser('map.osm')
print('Done')

Done


In [43]:
print("No of nodes - " , len(node_data))
print("No of edges - " , len(G))

No of nodes -  2074
No of edges -  632


In [44]:
# %%time
M = nx.Graph()
M.add_weighted_edges_from(G)

In [45]:
len(M.nodes()),len(M.edges())

(587, 632)